Notebook for scraping example test cases for each question

In [1]:
import pandas as pd
import time
import sys
import ast
import re
sys.path.append('..')
from utils import test_cases_from_slug

In [58]:
data = pd.read_csv('data/with_snippets/leetcode_hard_with_snippets_uncontaminated.csv')

In [121]:
import re

def extract_examples(description):
    inputs = [l for l in description.split('\n') if l.startswith('Input')]
    outputs = [l.strip('Output: ') for l in description.split('\n') if l.startswith('Output')]

    examples = []

    for input_str, output_str in zip(inputs, outputs):
        kwargs_str = re.sub(r'\s+', '', input_str)
        kwargs_pairs = re.findall(r"(\w+)\s*=\s*([-\w\[\],]+)(?:,|$)", kwargs_str)
        kwargs = {k: v for k, v in kwargs_pairs}
        output = output_str
        examples.append((kwargs, output))

    return examples

# Test the function with the provided problem description
examples = extract_examples(data.iloc[4]['description'])
for example in examples:
    print("kwargs:", example[0])
    print("output:", example[1])
    print()

kwargs: {'nums1': '[1,0,1]', 'nums2': '[0,0,0]', 'queries': '[[1,1,1],[2,1,0],[3,0,0]]'}
output: [3]

kwargs: {'nums1': '[1]', 'nums2': '[5]', 'queries': '[[2,0,0],[3,0,0]]'}
output: [5]



In [122]:
data['example_test_cases'] = None
for ind, row in data.iterrows():
    print(ind)
    examples = extract_examples(row['description'])
    data.at[ind, 'example_test_cases'] = examples

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40


In [123]:
data.to_csv('data/with_snippets/leetcode_hard_with_snippets_uncontaminated_tests.csv')

In [ ]:
data['example_test_cases'] = None
for ind, row in data.iterrows():
    print(ind)
    slug = row['question_slug']
    test_cases = test_cases_from_slug(slug)
    data.at[ind, 'example_test_cases'] = test_cases
    time.sleep(5)

In [51]:
data.iloc[6]['example_test_cases']

['[[3,4,2,1],[4,2,3,1],[2,1,0,0],[2,4,0,0]]',
 '[[3,4,2,1],[4,2,1,1],[2,1,1,0],[3,4,1,0]]',
 '[[2,1,0],[1,0,0]]']

In [52]:
function_name_regex = r"(?<=def\s)\w+"
no_args = 0
data['example_test_cases_parsed'] = None
for ind, row in data.iterrows():
    statements = []
    for test in row['example_test_cases']:
        args = test.split('\n')[:-1]
        if len(args) == 0:
            no_args += 1
        expected_val = test.split('\n')[-1]
        entry_point = re.search(function_name_regex, row['python3_snippet']).group(0)
        statement = f"assert {entry_point}({', '.join(args)}) == {expected_val}"
        statements.append(statement)
    #print(statements)
    data.at[ind, 'example_test_cases_parsed'] = statements
print(no_args)

21


In [21]:
data.to_csv('data/with_snippets/leetcode_hard_with_snippets_uncontaminated_cleaned_tests.csv')